# Fasion Mnist Convolution

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

import gzip
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
%config InlineBackend.figure_format = 'retina'

### read dataset

In [4]:
with gzip.open('./data/fashion-mnist/train-images-idx3-ubyte.gz', 'rb') as f:
    mnist_data = np.frombuffer(f.read(), np.uint8, offset=16)
    mnist_data = mnist_data.reshape(-1, 784)

In [5]:
mnist_data = mnist_data / 255
pd.DataFrame(mnist_data).head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.003922,0.0,0.0,0.000000,0.000000,...,0.466667,0.447059,0.509804,0.298039,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.086275,...,0.000000,0.000000,0.003922,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.129412,0.376471,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# 0 티셔츠, 1 바지, 2 스웨터, 3 드레스, 4 코트, 5 샌들, 6 셔츠, 7스니커즈, 8 가방, 9 앵클부츠
with gzip.open('./data/fashion-mnist/train-labels-idx1-ubyte.gz', 'rb') as f:
    mnist_label = np.frombuffer(f.read(), np.uint8, offset=8)
    
mnist_label = mnist_label.astype(int)
mnist_label

array([9, 0, 0, ..., 3, 0, 5])

### Train Test Split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(mnist_data, mnist_label, stratify=mnist_label, train_size=5000, test_size=500, random_state=0)


### generate tensor

In [31]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

### reshape

In [32]:
X_train = X_train.reshape(len(X_train), 1, 28, 28)
X_test = X_test.reshape(len(X_test), 1, 28, 28)

### generate dataset

In [34]:
train_dataset = TensorDataset(X_train, y_train)

data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

### Modeling

In [37]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)    

In [38]:
model = Net()

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(1000):
    total_loss = 0
    for batch_x, batch_y in data_loader:
        optimizer.zero_grad()
        
        result = model(batch_x)
        batch_loss = criterion(result, batch_y)
        batch_loss.backward()
        optimizer.step()
        
        total_loss += batch_loss.item()
    
    if (epoch+1) % 100 == 0:
        print(epoch, total_loss)

99 9.692848794162273
199 6.471020822995342
299 0.05260570508107776
399 1.503674733627122
499 0.031176962755125714
599 0.04032901069876971
699 0.0008574230685098883
799 0.0014609670683967124
899 4.5617788330787334e-05
999 0.0006577626680837056


### Validation

In [40]:
X_test, y_test = Variable(X_test), Variable(y_test)

result = torch.max(model(X_test).data, 1)[1]

accuracy = sum(result.numpy() == y_test.data.numpy()) / len(y_test.data.numpy())
accuracy

0.872